In [31]:
import cv2
import numpy as np
import csv
import time

In [32]:
VIDEO_PATH = "trafico01.mp4"
FPS = 25
SHOW_VIDEO = True
SAVE_RESULTS = True
RESIZE_W, RESIZE_H = 960, 540
ALPHA = 0.3          # suavizado velocidad
SMOOTH_FRAMES = 8     # suavizado centroides
MIN_AREA = 500

LINES = [
    (480, 850, 580, 850),
    (610, 850, 720, 850),
    (990, 750, 1090, 750),
    (1320, 900, 1470, 900),
    (1180, 640, 1210, 640),
    (1420, 730, 1500, 730),
    (1630, 770, 1770, 770),
]


In [33]:
def scale_lines(lines, orig_w, orig_h, new_w, new_h):
    """Escala las líneas de conteo a la nueva resolución"""
    sx, sy = new_w / orig_w, new_h / orig_h
    return [{"p1": (int(x1 * sx), int(y1 * sy)),
             "p2": (int(x2 * sx), int(y2 * sy)),
             "count": 0,
             "cars": set()} for (x1, y1, x2, y2) in lines]

def line_crossed(cx, cy, line, tol=10):
    """Devuelve True si el punto (cx, cy) cruza una línea"""
    x1, y1 = line["p1"]
    x2, y2 = line["p2"]
    return x1 <= cx <= x2 and abs(cy - y1) <= tol

def find_car(cx, cy, tracks, frame, max_dist=50, max_lost=5):
    """Asocia centroides a coches existentes"""
    for cid, pts in tracks.items():
        if not pts: continue
        x_last, y_last, f_last = pts[-1]
        if np.hypot(cx - x_last, cy - y_last) < max_dist and frame - f_last <= max_lost:
            return cid
    return None

def smooth_centroid(car_id, cx, cy, frame, tracks, N=SMOOTH_FRAMES):
    """Promedia los últimos N centroides"""
    tracks.setdefault(car_id, []).append((cx, cy, frame))
    tracks[car_id] = tracks[car_id][-N:]
    pts = np.array(tracks[car_id])
    return int(pts[:,0].mean()), int(pts[:,1].mean())

def calc_speed(track, fps):
    """Velocidad media en píxeles/segundo"""
    if len(track) < 2: return 0
    x0, y0, f0 = track[0]
    x1, y1, f1 = track[-1]
    d = np.hypot(x1 - x0, y1 - y0)
    t = (f1 - f0) / fps
    return d / t if t > 0 else 0

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
ret, f1 = cap.read()
ret, f2 = cap.read()

orig_h, orig_w = f1.shape[:2]
lines = scale_lines(LINES, orig_w, orig_h, RESIZE_W, RESIZE_H)

tracks, speeds = {}, {}
next_id, frame_id = 0, 0
start = time.time()

while ret:
    frame_id += 1
    f1 = cv2.resize(f1, (RESIZE_W, RESIZE_H))
    f2 = cv2.resize(f2, (RESIZE_W, RESIZE_H))

    # Detección simple por diferencia de frames
    diff = cv2.absdiff(cv2.cvtColor(f1, cv2.COLOR_BGR2GRAY),
                       cv2.cvtColor(f2, cv2.COLOR_BGR2GRAY))
    _, mask = cv2.threshold(cv2.blur(diff, (5,5)), 40, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        if cv2.contourArea(c) < MIN_AREA:
            continue
        x, y, w, h = cv2.boundingRect(c)
        cx, cy = x + w//2, y + h//2

        cid = find_car(cx, cy, tracks, frame_id)
        if cid is None:
            cid = next_id; next_id += 1
            tracks[cid] = []

        cx, cy = smooth_centroid(cid, cx, cy, frame_id, tracks)
        v = calc_speed(tracks[cid], FPS)
        speeds[cid] = ALPHA*v + (1-ALPHA)*speeds.get(cid, v)

        # Dibujar rectángulo y velocidad
        cv2.rectangle(f1, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(f1, f"{speeds[cid]:.1f} px/s", (x, y-8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,0), 2)

        # Conteo de líneas
        for line in lines:
            if line_crossed(cx, cy, line) and cid not in line["cars"]:
                line["count"] += 1
                line["cars"].add(cid)

    # Dibujar líneas
    for line in lines:
        color = (0,0,255) if line["count"] else (0,255,0)
        cv2.line(f1, line["p1"], line["p2"], color, 3)
        cv2.putText(f1, str(line["count"]), (line["p1"][0], line["p1"][1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    if SHOW_VIDEO:
        cv2.imshow("Conteo y Velocidad (px/s)", f1)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    f1 = f2
    ret, f2 = cap.read()

cap.release()
cv2.destroyAllWindows()


In [35]:
for i, line in enumerate(lines, 1):
    print(f"Línea {i}: {line['count']} coches")


Línea 1: 3 coches
Línea 2: 4 coches
Línea 3: 5 coches
Línea 4: 17 coches
Línea 5: 0 coches
Línea 6: 4 coches
Línea 7: 6 coches
